# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [ ]:
import os

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

In [ ]:
"""A simple edit of the original yfinance tool to make it less restrictive."""

from typing import Iterable, Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.documents import Document
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field
from requests.exceptions import HTTPError, ReadTimeout
from urllib3.exceptions import ConnectionError

from langchain_community.document_loaders.web_base import WebBaseLoader


class YahooFinanceNewsInput(BaseModel):
    """Input for the YahooFinanceNews tool."""

    query: str = Field(description="company ticker query to look up")


class YahooFinanceNewsTool(BaseTool):
    """Tool that searches financial news on Yahoo Finance."""

    name: str = "yahoo_finance_news"
    description: str = (
        "Useful for when you need to find financial news "
        "about a public company. "
        "Input should be a company ticker. "
        "For example, AAPL for Apple, MSFT for Microsoft."
    )
    top_k: int = 10
    """The number of results to return."""

    args_schema: Type[BaseModel] = YahooFinanceNewsInput

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """
        Use the Yahoo Finance News tool.

        Args:
            query: Company ticker symbol (e.g., 'AAPL' for Apple).
            run_manager: Optional callback manager.

        Returns:
            str: Formatted news results or error message.
        """
        try:
            import yfinance
        except ImportError:
            raise ImportError(
                "Could not import yfinance python package. "
                "Please install it with `pip install yfinance`."
            )
        company = yfinance.Ticker(query)

        try:
            if company.isin is None:
                return f"Company ticker {query} not found."
        except (HTTPError, ReadTimeout, ConnectionError):
            return f"Company ticker {query} not found."

        links = []

        try:
            links = [
                n["content"]["canonicalUrl"]["url"]
                for n in company.news
                if n["content"]["contentType"] == "STORY"
            ]
            # print(links)

        except (HTTPError, ReadTimeout, ConnectionError):
            if not links:
                return f"No news found for company that searched with {query} ticker."
        if not links:
            return f"No news found for company that searched with {query} ticker."
        loader = WebBaseLoader(web_paths=links)
        docs = loader.load()
        result = self._format_results(docs, query)
        if not result:
            return f"No news found for company that searched with {query} ticker."
        return result

    @staticmethod
    def _format_results(docs: Iterable[Document], query: str) -> str:
        doc_strings = [
            "\n".join([doc.metadata["title"], doc.metadata.get("description", "")])
            for doc in docs
            # if query in doc.metadata.get("description", "")
            # or query in doc.metadata["title"]
        ]
        return "\n\n".join(doc_strings)


In [ ]:
tools = [YahooFinanceNewsTool()]

In [ ]:
tool = YahooFinanceNewsTool()
tool.args

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [ ]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [ ]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a clear and succint financial strategy with precise numbers and allocations.
"""

In [ ]:
# Request from the client
request = "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [ ]:
Markdown(response.content)

## Simple Agent with Yahoo Finance News

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

In [ ]:
# Create a list of tools and a dictionnary of tool functions by name
tools = [YahooFinanceNewsTool()]
tools_by_name = {tool.name: tool for tool in tools}


In [ ]:
user_question = (
    # "I'm 25 year old and have $1,000 saved. which US stocks should I invest into?"
    "How does Microsoft feels today comparing with Nvidia?"
)
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you a question, and you will give them financial advice.
Then based on the stocks you advise, use the Yahoo Finance tool to get news if it's worth buying currently.
Give clear investment advice at the end. Do not assess risk.
"""

task_str = f"User question: {user_question}"

fa_model = base_model.bind_tools(tools)

In [ ]:
# Message list for the financial assistant model
messages = [
    SystemMessage(FINANCE_TOOL_PROMPT),
    HumanMessage(task_str),
]

# Invoke the financial assistant model with the messages
fa_output = fa_model.invoke(messages)

# If the financial assistant model made tool calls, invoke the tool
if fa_output.tool_calls:
    news_list = []
    id_list = []
    for tool_call in fa_output.tool_calls:
        tool = tools_by_name[tool_call["name"]]
        news = tool.invoke(tool_call["args"])

        display(Markdown(f"**Yahoo Finance news**: {news}"))

        news_list.append(news)
        id_list.append(tool_call["id"])

    # Combine the retrieved documents into a single string
    news_str = news
    # Message list with the retrieved documents for the base model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        HumanMessage(task_str),
        fa_output,
        *[
            ToolMessage(news_str, tool_call_id=tool_call_id)
            for (news_str, tool_call_id) in zip(news_list, id_list)
        ],
    ]

    # Invoke the base model with the messages
    fa_output_final = fa_model.invoke(messages)


Markdown(fa_output_final.content)

## Agentic system

We will now create a simple agentic system consisting of three agents using LangChain.
This system will help us perform a more elaborate financial analysis by including:

- Client Interface Agent: Rephrases the user’s prompt to improve the quality of the financial analyst’s response.

- Financial Analyst: Similar to the first part; provides financial advice based on the refined prompt.

- Risk Advisor: Assesses the risk associated with the advice given by the financial analyst.

In [ ]:
from typing import Literal

from langchain_core.messages import SystemMessage
from langgraph.constants import Send
from langgraph.graph import END
from langgraph.types import Command

## LangGraph Workflow and State

**LangGraph** helps us manage communication between our agents efficiently by defining a `State` class that will convey information from a node to the next during execution.

We keep our state simple by including only two attributes, but it's possible to include more:
- Messages: The ongoing conversation chain
- Analyses: Research findings from our agents

We use Python's dataclasses with special annotations (`Annotated`) to define how the state attributes should be updated throughout the workflow.

In [ ]:
import operator
from dataclasses import dataclass, field
from typing import Annotated

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

In [ ]:
@dataclass(kw_only=True)
class State:
    """Graph state for the financial analysis workflow."""

    messages: Annotated[list[BaseMessage], add_messages] = field(default_factory=list)
    yahoo_finance_news: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )
    risk_analysis: Annotated[list[BaseMessage], add_messages] = field(
        default_factory=list
    )

## Orchestrator

In [ ]:
ORCHESTRATOR_PROMPT = """
You are going to reformulate the user question to make it more precise for a financial analyst.
"""
# Create the orchestrator model model from the base model with tool binding
bra_model = base_model.bind_tools(tools)

In [ ]:
# Define the CIA orchestrator with the next node options
def orchestrator_node(
    state: State,
) -> Command[Literal["financial_analyst", "risk_analyst", END]]:
    """Orchestrator that generates a plan for the financial analysis."""
    display(Markdown(f"**Client request received**: {state.messages[-1].content}"))

    # Message list for the orchestrator model
    messages = [
        SystemMessage(ORCHESTRATOR_PROMPT),
        *state.messages,
    ]

    # Invoke the CIA model
    orchestrator_output = base_model.invoke(messages)

    display(Markdown(f"**CIA Response:** {orchestrator_output.content}"))

    return Command(
        # Update the state messages with the CIA response
        update={"messages": orchestrator_output.content},
        # Go to worker nodes if the request is in scope, otherwise end the workflow
        goto=["financial_analyst", "risk_analyst"],
    )

## Worker Nodes

### Financial analyst Node

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

In [ ]:
FINANCE_TOOL_PROMPT = """
You are a Financial Analyst. The client will ask you a question, and you will give them financial advice.
Then based on the stocks you advise, use the Yahoo Finance tool to get news if it's worth buying currently.
Give clear investment advice at the end. Do not assess risk.
"""

# Create the financial assistant model from the base model with tool binding
fa_model = base_model.bind_tools(tools)

In [ ]:
# Define the worker node and the next node options
def financial_analyst_node(state: State) -> Command[Literal["synthesizer"]]:
    """Given a user question, get financial advice from Yahoo Finance news."""
    display(Markdown("**Giving financial advice without risk information**"))
    # Message list for the financial assistant model
    messages = [
        SystemMessage(FINANCE_TOOL_PROMPT),
        *state.messages,
    ]

    # Invoke the financial assistant model with the messages
    fa_output = fa_model.invoke(messages)

    # If the financial assistant model made tool calls, invoke the tool
    if fa_output.tool_calls:
        news_list = []
        id_list = []
        for tool_call in fa_output.tool_calls:
            tool = tools_by_name[tool_call["name"]]
            news = tool.invoke(tool_call["args"])

            # display(Markdown(f"**Yahoo Finance news**: {news}"))

            news_list.append(news)
            id_list.append(tool_call["id"])

        # Message list with the retrieved documents for the base model
        messages = [
            SystemMessage(FINANCE_TOOL_PROMPT),
            HumanMessage(task_str),
            fa_output,
            *[
                ToolMessage(news_str, tool_call_id=tool_call_id)
                for (news_str, tool_call_id) in zip(news_list, id_list)
            ],
        ]

        # Invoke the base model with the messages
        fa_output = fa_model.invoke(messages)

    # Update the state analyses with the financial analyst output content and go to the synthesizer node
    # NOTE: To update `analyses` you should return a list
    return Command(
        update={"yahoo_finance_news": [fa_output.content]},
        goto="synthesizer",
    )

### Risk analyst Node

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
RISK_ANALYST_PROMPT = """
You are a Risk Advisor.
Evaluate the analyst's advice from a risk perspective. Offer any cautions and ways to reduce risk, don't repeat the analyst's advice.
"""

In [ ]:
# Define the synthesizer node and the next node options
def risk_analyst_node(state: State) -> Command[Literal[END]]:
    """Given the user prompt perform risk analysis."""
    display(Markdown("**Performing risk analysis.**"))

    # Access the previous responses
    finance_analysis = state.yahoo_finance_news

    # Combine the previous messages into a single string
    complete_analyses = "The financial advisor's analysis:" + "\n\n---\n\n".join(
        finance_analysis
    )

    # Message list for the risk analyst
    messages = [
        SystemMessage(RISK_ANALYST_PROMPT),
        *state.messages,
        AIMessage(complete_analyses),
    ]

    # Invoke the base model with the messages
    risk_analyst_output = base_model.invoke(messages)

    # Update the state messages with the risk analyst's output content and go to the synthesizer node
    return Command(
        update={"risk_analysis": [risk_analyst_output.content]},
        goto="synthesizer",
    )

### Synthesiser node

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
SYNTHESISER_PROMPT = """
Your are a financial analyst, take the general financial analyst's insights and the risk analyst's insights and combine them to generate comprehensive financial advise for the user.
"""

In [ ]:
# Define the synthesizer node and the next node options
def synthesizer_node(state: State) -> Command[Literal[END]]:
    """Synthesize full report from research analyses."""
    display(Markdown("**Synthesizing messages from the two analysts.**"))

    # Access the previous responses
    finance_analysis = state.yahoo_finance_news

    risk_analysis = state.risk_analysis

    # Combine the research analyses into a single string
    financial_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in finance_analysis]
    )
    risk_analysis_str = "risk analysis: " + "\n\n---\n\n".join(
        [item.content for item in risk_analysis]
    )

    # Message list for the RSA model
    messages = [
        SystemMessage(SYNTHESISER_PROMPT),
        AIMessage(financial_analysis_str),
        AIMessage(risk_analysis_str),
    ]

    # Invoke the base model with the messages
    synth_output = base_model.invoke(messages)

    # Update the state messages with the RSA output content and end the workflow
    return Command(
        update={"messages": synth_output},
        goto=END,
    )

## Building the Workflow Graph

Now that our nodes and communication flow are defined, we can build the graph!

In [ ]:
from IPython.display import Image
from langgraph.graph import StateGraph

In [ ]:
# Create a state graph builder
graph_builder = StateGraph(State)

# Define the entry point
graph_builder.set_entry_point("orchestrator")

# Add the nodes
graph_builder.add_node("orchestrator", orchestrator_node)
graph_builder.add_node("financial_analyst", financial_analyst_node)
graph_builder.add_node("risk_analyst", risk_analyst_node)
graph_builder.add_node("synthesizer", synthesizer_node)

# The edges are defined by the commands !

# Compile the workflow
app = graph_builder.compile()

Let's visualize our Financial Analyst graph. Note that because the number of `"worker"` nodes is generated dynamically, it shows up as a single node in the image.

In [ ]:
# display(Image(app.get_graph().draw_mermaid_png()))
#

## Running the Workflow

Now that our workflow is built, let's test it! Once again, we can run it with `.invoke()`.

In [ ]:
request = (
    "I am a 25 year old and have $1,000 saved. which US stocks should I invest into?"
)

# Invoke the workflow with the client request
final_state = app.invoke({"messages": request})

In [ ]:
Markdown(final_state["messages"][-1].content)